<a href="https://colab.research.google.com/github/cbh4635/DL_studty/blob/main/2d_object_detection_Faster_R-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

# GPU 사용 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device:", device)

Using device: cuda


# PyTorch기반 주요 Pre-trained 모델 라이브러리
1. torchvision
- pytorch 공식 제공 라이브러리, 안전성이 높으나 모델 업데이트가 늦어 제공되는 모델 수가 적음
2. timm (Torch IMage Models)
- pytorch기반 서드파티 라이브러리, 모델 업데이트가 빨라 많은 최신 백본을 제공함
3. mmdet (MMDetection)
- Object detection용 라이브러리, 최신 detection 모델이 많고 모듈화 구조로 다양한 백본과 Head를 조합하여 모델개발 용이
4. Hugging Face Transformers
- 여러 분야의 SotA 모델들이 빠르게 공유되는 오픈소스 라이브러리 (사내망 사용불가 - 보안예외 필요)

In [ ]:
import torchvision
# Faster R-CNN COCO pre-trained 모델 로드
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)
# model = fasterrcnn_resnet50_fpn(pretrained=True) - 이것도 가능

model.to(device)
model.eval()

# MS COCO 데이터셋 클래스 이름 가져오기
COCO_CATEGORIES = weights.meta["categories"]

In [6]:
print(COCO_CATEGORIES)
len(COCO_CATEGORIES) # 실제 사용되는 class 수는 80개

['__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


91

## 입력 이미지 다운로드

In [ ]:
!mkdir /content/data
!wget -O ./data/beatles01.jpg https://raw.githubusercontent.com/chulminkw/DLCV/master/data/image/beatles01.jpg

# OpenCV
- 인텔이 개발한 Computer Vision 라이브러리
- Python 이외에도 다양한 language 지원 (C++,C#,Java등)

In [ ]:
import cv2 # OpenCV
import matplotlib.pyplot as plt

fname = './data/beatles01.jpg'

# OpenCV로 이미지 읽기 'cv2.imread()'
img = cv2.imread(fname)
print('image shape:', img.shape) # (H,W,C)

In [ ]:
img_bgr = img
plt.figure(figsize=(8, 8))
plt.imshow(img_bgr)

# 'cv2.cvtColor()'를 이용하여 BGR -> RGB 변환
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)

In [ ]:
from torchvision.transforms.functional import to_tensor
# 'to_tensor': 이미지 데이터를 [0,1] 사이 float값으로 Nomalization하고 tensor로 변환
img_tensor = to_tensor(img_rgb).to(device)

with torch.no_grad(): # gradient 계산 안함
    outputs = model([img_tensor])  # list[dict]
out = outputs[0]

boxes = out["boxes"].cpu()
labels = out["labels"].cpu()
scores = out["scores"].cpu()

# 결과 확인
print('box shape:', boxes.shape)
print('label shape:', labels.shape)
print('score shape:', scores.shape)

## 모델 추론결과 시각화 함수 정의 (이미지)

In [13]:
def draw_boxes_on_image(img_input, boxes, labels, scores, score_threshold=0.5):
    """
    img_input : OpenCV 이미지 (np.ndarray)
    boxes     : (N, 4) tensor [x1, y1, x2, y2]
    labels    : (N,) tensor (int, COCO class index)
    scores    : (N,) tensor (float)
    """
    img = img_input.copy()
    h, w = img.shape[:2]
    box_color = (0, 255, 0) # green

    for box, label, score in zip(boxes, labels, scores):
        if score < score_threshold:
            continue

        # 박스 설정
        x1, y1, x2, y2 = box.int().tolist() # int로 변환(pixel 위치), tensor를 배열로 변경
        cv2.rectangle(img, (x1, y1), (x2, y2), box_color, 2)

        # 라벨 텍스트 설정
        class_name = COCO_CATEGORIES[label]
        text = f"{class_name}: {score:.2f}"

        # 텍스트 배경 박스 설정
        (tw, th), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(img, (x1, y1 - th - baseline), (x1 + tw, y1), box_color, -1)
        cv2.putText(img, text, (x1, y1 - baseline),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    return img

In [ ]:
# 박스 그리기
img_with_boxes = draw_boxes_on_image(img_rgb, boxes, labels, scores, score_threshold=0.5)

# 모델 결과 확인
plt.figure(figsize=(12, 12))
plt.imshow(img_with_boxes)

## 입력 비디오 다운로드

In [ ]:
!wget -O ./data/Jonh_Wick_small.mp4 https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true

In [ ]:
vname = '/content/data/Jonh_Wick_small.mp4'

# 'cv2.VideoCapture()'를 이용하여 비디오 객체를 생성
cap = cv2.VideoCapture(vname)
# 'cap.get()'은 비디오 객체의 다양한 속성값을 가져올 수 있음
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('총 Frame 갯수:', frame_cnt)

## 모델 추론결과 시각화 함수 정의 (비디오)

In [20]:
def run_detection_on_video(model, video_path, output_path, score_threshold=0.5, speed_factor=1):
    """
    model        : 추론 모델
    video_path   : 입력 비디오 경로
    output_path  : 출력 비디오 저장 경로
    score_threshold : 바운딩 박스 신뢰도 threshold
    speed_factor    : 출력 비디오 실행 배속 (fps*speed_factor)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS) # 입력 비디오의 fps
    w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 입력 비디오의 width
    h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 입력 비음오의 height

    codec = cv2.VideoWriter_fourcc(*'mp4v') # or 'XVID'
    out_writer = cv2.VideoWriter(output_path, codec, fps*speed_factor, (w, h))

    frame_idx = 0
    while True:
        # 'cap.read()'는 마지막 Frame까지 차례대로 비디오를 읽음
        has_frame, frame_bgr = cap.read() # [bool, image]
        if not has_frame:
            print('No frame available for video')
            break

        # BGR -> RGB
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        frame_tensor = to_tensor(frame_rgb).to(device)

        with torch.no_grad():
            outputs = model([frame_tensor])
        out = outputs[0]

        boxes = out["boxes"].cpu()
        labels = out["labels"].cpu()
        scores = out["scores"].cpu()

        frame_bgr = draw_boxes_on_image(frame_bgr, boxes, labels, scores, score_threshold)

        # 저장시 이미지를 BGR 형태로 입력해야 RGB로 변환되어 저장됨
        out_writer.write(frame_bgr)
        frame_idx += 1
        print("Frame: ", frame_idx)

    cap.release()
    out_writer.release()
    print(f"Saved video with detections: {output_path}")

In [ ]:
import os
os.makedirs("outputs_videos", exist_ok=True)

in_path  = vname
out_path = os.path.join("outputs_videos", f"det_{os.path.basename(vname)}")

run_detection_on_video(
    model=model,
    video_path=in_path,
    output_path=out_path,
    score_threshold=0.7,
    speed_factor=0.3
)